In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 999

n = 200
ordersid = np.random.randint(1, 500, size=n)
customerid = np.random.randint(583, 19573, size=n)
dates = pd.date_range('2018-11-29 12:00:00', periods=n, freq=None)
productid = np.random.randint(1, 10, size=n)
products = pd.Series([1,2,3,4,5,6,7,8,9,10])
price = pd.Series([10, 20, 100, 150, 30, 400, 5, 950, 523, 14000])

In [2]:
prod_pric = pd.DataFrame({'ProductId': products, 'Price':price})

In [3]:
pro_ord_pri = pd.DataFrame({'ProductId': productid, 'OrderId':ordersid})

In [4]:
order_lines = pd.merge(pro_ord_pri, prod_pric, how = 'left', on = 'ProductId')

In [5]:
orders = pd.DataFrame({'OrderId' : ordersid, 'CustomerId': customerid, 'DateTime': dates})

- orders содержит информацию о заказах: идентификатор заказа, идентификатор покупателя, время заказа:

OrderId,CustomerId,DateTime

5,583,2017-01-01 15:03:17

13,900,2019-02-05 05:02:59

69,19573,2018-11-03 23:59:59

...

- order_lines содержит информацию о линиях чеков: идентификатор продукта, идентификатор заказа, цену продукта

ProductId,OrderId,Price

5873,5,3026.0

7265,5,573.0

9675,5,159.0

5873,6,2999.0

13,6,57.0

...

1. Постройте отчёт (таблица в виде DataFrame), где видны

самые популярные за последний месяц продукты

суммарная выручка (за последний месяц) по каждому такому продукту

средний чек заказов (за последний месяц), в которых есть такие продукты

### Поехал код

In [6]:
data = pd.merge(orders, order_lines, how = 'left', on = 'OrderId')

In [7]:
from datetime import datetime, timedelta
data = data.set_index(['DateTime'])
curr_date = datetime.today()
#не совсем понятно, что подразумевается под "за последний месяц". За 30 дней? Или за текущий неполный месяц, например за июнь?
#или же за прошлый полный месяц? решено остановиться на варианте за последние 30 дней.
last_date = curr_date - timedelta(days = 30)
data_month = data.loc[last_date:curr_date]
data_month

,OrderId,CustomerId,ProductId,Price
DateTime,,,,
2019-05-18 12:00:00,218,5481,4,150
2019-05-19 12:00:00,423,6693,4,150
2019-05-20 12:00:00,294,7059,2,20
2019-05-20 12:00:00,294,7059,5,30
2019-05-21 12:00:00,487,11790,9,523
2019-05-22 12:00:00,459,9229,4,150
2019-05-22 12:00:00,459,9229,8,950
2019-05-23 12:00:00,395,6602,6,400
2019-05-24 12:00:00,204,7241,4,150


In [8]:
#в качестве самый популярных продуктов были определены топ-5 продуктов, которые встречаются чаще всего в датафрейме
top_5_prod = data_month['ProductId'].value_counts().iloc[0:5]
top_5_prod = pd.DataFrame(top_5_prod).reset_index().rename(columns = {'index':'ProductId','ProductId':'Frequency'})
top_5_prod = pd.merge(top_5_prod, order_lines, how = 'left', on = 'ProductId')
top_5_prod

,ProductId,Frequency,OrderId,Price
0,5,8,330,30
1,5,8,346,30
2,5,8,237,30
3,5,8,318,30
4,5,8,386,30
5,5,8,55,30
6,5,8,304,30
7,5,8,442,30
8,5,8,401,30
9,5,8,276,30


In [9]:
#выручка определяется как произведение количества товара в заказах и цены товара
revenue = top_5_prod.drop_duplicates(subset = 'ProductId')
revenue['Revenue'] = revenue['Frequency'] * revenue['Price']
revenue = revenue.drop(['Frequency','OrderId','Price'], axis = 1)

C:\Users\HyperPC\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
pre_table = pd.pivot_table(data_month, 
                       values=['OrderId'], 
                       index=['ProductId'],
                       aggfunc = lambda x: x.value_counts().count())
pre_table = pre_table.rename(columns = {'OrderId':'PurchaseNum'})
top_5_prod = pd.merge(top_5_prod, pre_table, how = 'left', on = 'ProductId')
top_5_prod = pd.merge(top_5_prod, revenue, how = 'left', on = 'ProductId')
#fin_table = pd.pivot_table(top_5_prod, 
#                      values=['AverageBill'], 
#                       index=['ProductId','Revenue','OrderId'],
#                       aggfunc = np.mean)

In [13]:
pre_table

,PurchaseNum
ProductId,
1,1
2,3
3,4
4,6
5,8
6,4
7,1
8,3
9,6
